In [1]:
# def day4_parser(line: str) -> tuple:
#     pass
    

# def day4_part1(policies: list[tuple]) -> int:
#     pass


# def day4_part2(policies: list[tuple]) -> int:
#     pass

# run(day=3, parser=day4_parser, sep='\n', funcs=[day4_part1, day4_part2], example=True, actual=True)


# for i in range(1, 26):
#     with open(f'data/day{i:02}.txt', 'a') as f:
#         f.write('')

In [76]:
import timeit
from itertools import permutations, combinations, product, chain, islice
from math import ceil, floor

#Dict, Tuple, Set, List, Iterator, Optional, Union, Sequence

# from __future__  import annotations
# from collections import Counter, defaultdict, namedtuple, deque
# from itertools   import permutations, combinations, product, chain
from functools   import partial  #lru_cache, reduce
# from typing      import Dict, Tuple, Set, List, Iterator, Optional, Union, Sequence
# from contextlib  import contextmanager

# import operator
# import math
# import ast
# import sys
import re


from more_itertools import minmax

def first(iterable, default=None) -> object:
    "Return first item in iterable, or default."
    return next(iter(iterable), default)


def quantify(iterable, pred=bool) -> int:
    "Count the number of items in iterable for which pred is true."
    return sum(1 for item in iterable if pred(item))

In [7]:
def data(path: str, parser=str, sep: str='\n') -> list:
    "Load input textfile into sections separated by `sep`, and apply `parser` to section."   
    with open(path, 'r') as f:
        sections = f.read().rstrip().split(sep)
        return list(map(parser, sections))


def run(day: int, funcs: list=[], example: bool=True, actual: bool=False, parser=str, sep: str='\n') -> None:
    """Run a day's example (and/or actual) input on a list of callables for each part i.e. [part1, part2]. 
    While working can use [part1, lambda _: None]."""
    if example:
        puzzle_input = data(path=f'data/example/day{day:02}.txt', parser=parser, sep=sep)
        print('Part 1 (example):', funcs[0](puzzle_input), end='  ')
        print('\tPart 2 (example):', funcs[1](puzzle_input))
    
    if actual:
        puzzle_input = data(path=f'data/day{day:02}.txt', parser=parser, sep=sep)
        print('Part 1 (actual):', funcs[0](puzzle_input), end='  ')
        print('\tPart 2 (actual):', funcs[1](puzzle_input))


def test_alternatives(day: int, funcs: list=[], runs: int=10, parser=str, sep: str='\n') -> None:
    "Test alternative approaches on a specific part of a day's actual input."
    puzzle_input = data(path=f'data/day{day}.txt', parser=parser, sep=sep)
    
    print('Func Result:', end='\t')
    for func in funcs:  # Check we get the expected 
        print(func(puzzle_input), end=' ')

    print('\nFunc Runtime:', end='\t')
    for func in funcs:  # Compare each functions runtime
        print(f'{timeit.timeit("func(puzzle_input)", number=runs, globals=locals()):.3f}', end='  ')

## Day 01: Report Repair
- Part 1: find two numbers that sum to 2020, return their product
- Part 2: find three numbers that sum to 2020, return their product

In [4]:
def day1_part1(nums: list[int]) -> int:
    for x in nums:
        for y in nums:
            if x + y == 2020:
                return x * y


def day1_part2(nums: list[int]) -> int:
    for x in nums:
        for y in nums:
            for z in nums:
                if x + y + z == 2020:
                    return x * y * z


run(day=1, parser=int, sep='\n', funcs=[day1_part1, day1_part2], example=False, actual=True)

Part 1 (actual): 802011  	Part 2 (actual): 248607374


### Other Solutions

In [5]:
def day1_part1b(nums: list[int]) -> int: # still O(n^2) as above, but slighty faster
    for i, x in enumerate(nums):
        for y in islice(nums, i): # not using nums[i:] as that creates a copy each time (would make this O(n^3))
            if x + y == 2020:
                return x * y


def day1_part1c(nums: list[int]) -> int: # O(n^2)
    for x, y in combinations(nums, 2): 
        if x + y == 2020:
            return x * y


def day1_part1d(nums: list[int]) -> int: # O(n)
    seen = set()
    for x in nums:
        if (2020 - x) in seen:
            return (2020 - x) * x
        seen.add(x)


def day1_part1e(nums: list[int]) -> int: # O(n) the second loop is only run max twice (i.e. 2020 + 20 and 20 + 2020)
    nums = set(nums) # ideally input would already be a set
    return first(
        x * y 
        for x in nums 
        for y in nums.intersection({2020 - x})
        if x != y
        )


test_alternatives(day=1, funcs=[day1_part1, day1_part1b, day1_part1c, day1_part1d, day1_part1e], runs=1000, parser=int)

Func Result:	802011 802011 802011 802011 802011 
Func Runtime:	0.350  0.170  0.379  0.008  0.004  

In [39]:
def day1_part2b(nums: list[int]) -> int: # still O(n^3) as above, has faster wall time though
    for x, y, z in combinations(nums, 3): 
        if x + y + z == 2020:
            return x * y * z


def day1_part2c(nums: list[int]) -> int: # O(n^2), but slower wall time (on the test input/size)
    set_nums = set(nums)
    for x, y in combinations(nums, 2):
        if (2020 - x - y) in set_nums:
            return x * y * (2020 - x - y)


def day1_part2d(nums: list[int]) -> int: # O(n^2)
    nums = set(nums)
    return first(
        x * y * z 
        for x, y in combinations(nums, 2) 
        for z in nums.intersection({2020 - x - y})
        if x != y != z
        )


def day1_part2e(nums: list[int]) -> int: # O(N^2) time, O(1) space
    nums.sort()  # sort then use two pointers
    for i in range(len(nums)):
        l, r = i+1, len(nums)-1 # left starts at next index, right starts at last index
        while l < r:
            if (nums[l] + nums[r] + nums[i]) > 2020:
                r -= 1
            elif (nums[l] + nums[r] + nums[i]) < 2020:
                l += 1
            elif (nums[l] + nums[r] + nums[i]) == 2020:
                return nums[l] * nums[r] * nums[i]


test_alternatives(day=1, funcs=[day1_part2, day1_part2b, day1_part2c, day1_part2d, day1_part2e], runs=10, parser=int)

Func Result:	248607374 248607374 248607374 248607374 248607374 
Func Runtime:	0.037  0.020  0.000  0.006  0.001  

# Day 02: Password Philosophy
- Input Line: #-# character: password (i.e. 1-3 e: abcde)
- Part 1: find number of passwords with character count between low and high # 
- Part 2: find number of passwords with character at idx #(1) or idx #(2) (1-based indexes and XOR)

In [6]:
def day2_parser(line: str) -> tuple:
    "Given line '1-3 e: mdie', return (1, 3, 'e', 'mdie')"
    nums, char, pw = line.split()
    a, b = map(int, nums.split('-'))
    char = char.rstrip(':')
    return (a, b, char, pw)
    

def day2_part1(policies: list[tuple]) -> int:
    cnt = 0
    for policy in policies:
        lo, hi, char, pw = policy
        if lo <= pw.count(char) <= hi:
            cnt += 1
    return cnt


def day2_part2(policies: list[tuple]) -> int:
    cnt = 0
    for policy in policies:
        pos1, pos2, char, pw = policy
        one, two = pw[pos1-1], pw[pos2-1] # minus 1 as these positions are 1-based indexed
        if (one==char) != (two==char): # could use xor (^) instead
            cnt += 1
    return cnt


run(day=2, parser=day2_parser, sep='\n', funcs=[day2_part1, day2_part2], example=True, actual=True)

Part 1 (example): 2  	Part 2 (example): 1
Part 1 (actual): 477  	Part 2 (actual): 686


In [7]:
Policy = tuple[int, int, str, str] 
Policies = list[Policy]


def day2_parserb(line: str) -> Policy:
    "Given line '1-3 e: mdie', return (1, 3, 'e', 'mdie')"
    a, b, char, pw = re.findall(r'[^-:\s]+', line) # or r'(\d+)-(\d)+ (.): (.+)$'
    return (int(a), int(b), char, pw)


def day2_part1b(policies: Policies) -> int:
    def valid_password(policy: Policy) -> bool:
        lo, hi, char, pw = policy
        return lo <= pw.count(char) <= hi
    return quantify(policies, valid_password)


def day2_part2b(policies: Policies) -> int:
    def valid_password(policy: Policy) -> bool:
        pos1, pos2, char, pw = policy
        one, two = pw[pos1-1], pw[pos2-1]  # minus 1 as these positions are 1-based indexed
        return (one==char) ^ (two==char)  # could use != instead of XOR
    return quantify(policies, valid_password)


run(day=2, parser=day2_parserb, sep='\n', funcs=[day2_part1b, day2_part2b], example=True, actual=True)

Part 1 (example): 2  	Part 2 (example): 1
Part 1 (actual): 477  	Part 2 (actual): 686


# Day 3: Toboggan Trajectory
- Input is a map of freespaces (.) & trees (#) copied infinitely to the right
- Part 1: count trees landed on moving right then down (3,1 places starting from top left)
- Part 2: return product of tree counts on the following paths: (1,1), (3,1), (5,1), (7,1), (2,1)

In [19]:
def day3_helper(treemap: list[str], x_step: int, y_step: int):
    x = 0  # starting horizontal position
    y = 0  # starting vertical position
    tree_count = 0
    
    while y < len(treemap):
        row = treemap[y]
        if row[x] == '#':  # landed on a tree
            tree_count += 1
        x = (x + x_step) % len(row)  # wrap around the x-axis
        y += y_step  # move down the y-axis
    return tree_count


def day3_part1(treemap: list[str]) -> int:
    return day3_helper(treemap, 3, 1)


def day3_part2(treemap: list[str]) -> int:
    f = lambda x, y: day3_helper(treemap, x, y)
    return f(1, 1) * f(3, 1) * f(5, 1) * f(7, 1) * f(1, 2)


run(day=3, parser=str, sep='\n', funcs=[day3_part1, day3_part2], example=True, actual=True)

Part 1 (example): 7  	Part 2 (example): 336
Part 1 (actual): 191  	Part 2 (actual): 1478615040


In [20]:
def day3_helperb(treemap: list[str], x_step=3, y_step=1) -> int:
    return quantify(row[(x_step * y) % len(row)] == '#'
                    for y, row in enumerate(treemap[::y_step]))
    # aka:
    # cnt = 0
    # for y, row in enumerate(treemap[::y_step]):
    #     # x = (x_step * y) 
    #     # pos = (x_step * y) % len(row)
    #     # print(x, y, pos)
    #     cnt += 1 if row[(x_step * y) % len(row)] == '#' else 0
    # return cnt 

def day3_part1b(treemap: list[str]) -> int:
    return day3_helperb(treemap, 3, 1)


def day3_part2b(treemap: list[str]) -> int:
    f = lambda x, y: day3_helperb(treemap, x, y)
    return f(1, 1) * f(3, 1) * f(5, 1) * f(7, 1) * f(1, 2)


run(day=3, parser=str, sep='\n', funcs=[day3_part1b, day3_part2b], example=True, actual=True)

Part 1 (example): 7  	Part 2 (example): 336
Part 1 (actual): 191  	Part 2 (actual): 1478615040


# Day 4: Passport Processing
- Input is a series of passports seperated by double newlines, each passort has # fields (key:value pairs).
- Part 1: count how many passports have required fields.
- Part 2: count how many passports have required fields and ensure each field meets set criteria.

In [24]:
REQUIRED_FIELDS = {"byr", "ecl", "eyr", "hcl", "hgt", "iyr", "pid"}


def day4_parser(line: str) -> dict:
    """Create a dict from text ala 'ecl:gry eyr:2020' -> {'ecl':'gry', 'eyr':'2020'}"""
    return dict(kv.split(":") for kv in line.split())


def day4_part1(passports: list[dict]) -> int:
    # alt.: set(p).issuperset(REQUIRED_FIELDS)
    return quantify(passports, lambda p: p.keys() >= REQUIRED_FIELDS)


def day4_part2(passports: list[dict]) -> int: 
    between = lambda v, start, end: start <= int(v) <= end
    height = lambda v: (v.endswith("cm") and between(v[:-2], 150, 193)) or (
        v.endswith("in") and between(v[:-2], 59, 76)
    )
    
    eye_colour = lambda v: v in {"amb", "blu", "brn", "gry", "grn", "hzl", "oth"}
    hair_colour = lambda v: re.match("#[0-9a-f]{6}$", v)
    passport_id = lambda v: re.match("[0-9]{9}$", v)

    def valid_passport(p) -> bool:
        return p.keys() >= REQUIRED_FIELDS and all(
            (
                between(p["byr"], 1920, 2002),
                between(p["iyr"], 2010, 2020),
                between(p["eyr"], 2020, 2030),
                height(p["hgt"]),
                eye_colour(p["ecl"]),
                hair_colour(p["hcl"]),
                passport_id(p["pid"]),
            )
        )

    return quantify(passports, valid_passport)



run(day=4, parser=day4_parser, sep="\n\n", funcs=[day4_part1, day4_part2], example=True, actual=True)

Part 1 (example): 2  	Part 2 (example): 2
Part 1 (actual): 170  	Part 2 (actual): 103


# Day 5: Binary Boarding

![""](img/day5.png "day5 example")

In [129]:
def binary_partitioning(search_space: str, binary_choice):
    lb, ub = 0, (2**len(search_space))-1  # lower and upper bounds
    for item in search_space:
        if item != binary_choice:
            lb = ceil(lb/2) + ceil(ub/2)
        else:
            ub = floor(lb/2) + floor(ub/2)
    return min(lb, ub)


def day5_helper(seats: list[str]) -> list[int]:
    "Convert a list of seat strings to seat ids ala ['FBFBBFFRLR'] -> [357]."
    row = partial(binary_partitioning, binary_choice='F')
    col = partial(binary_partitioning, binary_choice='L')
    return [row(seat[:7]) * 8 + col(seat[7:]) for seat in seats]


def day5_part1(seats: list[str]) -> int:
    return max(day5_helper(seats))


def day5_part2(seats: list[int]) -> set:
    seat_ids = day5_helper(seats)
    all_seat_ids = set(i for i in range(*minmax(seat_ids)))
    return (all_seat_ids - set(seat_ids))


run(day=5, parser=str, sep='\n', funcs=[day5_part1, day5_part2], example=True, actual=True)

Part 1 (example): 357  	Part 2 (example): set()
Part 1 (actual): 888  	Part 2 (actual): {522}


In [155]:
def day5_helperb(seats: list[str], table=str.maketrans('FLBR', '0011')) -> list[int]:
    "Treat a seat description as a binary number; convert to int."
    return [int(s.translate(table), base=2) for s in seats]

def day5_part1(seats: list[str]) -> int:
    return max(day5_helper(seats))


def day5_part2(seats: list[int]) -> set:
    seat_ids = day5_helper(seats)
    all_seat_ids = set(i for i in range(*minmax(seat_ids)))
    return (all_seat_ids - set(seat_ids))


run(day=5, parser=str, sep='\n', funcs=[day5_part1, day5_part2], example=True, actual=True)

Part 1 (example): 357  	Part 2 (example): set()
Part 1 (actual): 888  	Part 2 (actual): {522}


# Day 6

In [ ]:
# Example 1
puzzle_input = '''abc

a
b
c

ab
ac

a
a
a
a

b'''

z = ["".join(set(i.replace('\n',''))) for i in puzzle_input.split('\n\n')]
print(len("".join(z)))

In [ ]:
# Part 1
with open('/content/drive/MyDrive/advent_of_code/day6.txt', 'r') as file:
    puzzle_input = file.read()

print(len("".join(["".join(set(i.replace('\n',''))) for i in puzzle_input.split('\n\n')])))

In [ ]:
# Example 2
from collections import Counter
puzzle_input = '''abc

a
b
c

ab
ac

a
a
a
a

b'''

puzzle_processing = [i.split('\n') for i in puzzle_input.split('\n\n')]

result = 0

for group in puzzle_processing:
    group_cnt = len(group)
    answers = Counter()

    for ans in ''.join(group):
        answers[ans] += 1
    
    for ans in answers.items():
        if ans[1] % group_cnt == 0:
            x += 1

print(result)

In [ ]:
# Part 2
from collections import Counterl

with open('/content/drive/MyDrive/advent_of_code/day6.txt', 'r') as file:
    puzzle_input = file.read()

puzzle_processing = [i.split('\n') for i in puzzle_input.split('\n\n')]
result = 0

for group in puzzle_processing:
    group_cnt = len(group)
    answers = Counter()

    for ans in ''.join(group):
        answers[ans] += 1
    
    for ans in answers.items():
        if ans[1] % group_cnt == 0:
            result += 1

print(result)

# Day 7

In [ ]:
class Graph:
    def __init__(self, edges):
        self.edges = edges
        self.graph_dict = {}
        for start, end in self.edges:
            if start in self.graph_dict:
                self.graph_dict[start].append(end)
            else:
                self.graph_dict[start] = [end]
        print("Graph dict:", self.graph_dict)

    def get_paths(self, start, end, path=[]):
        path = path + [start]

        if start == end:
            return [path]
        
        if start not in self.graph_dict:
            return []
        
        paths = []

        for node in self.graph_dict[start]:
            if node not in path:
                new_paths = self.get_paths(node, end, path) 
                for p in new_paths:
                    paths.append(p)
        
        return paths

    def get_shortest_path(self, start, end, path=[]):
        path = path + [start]

        if start == end:
            return path

        if start not in self.graph_dict:
            return None

        shortest_path = None
        for node in self.graph_dict[start]:
            sp = self.get_shortest_path(node, end, path)
            if sp:
                if shortest_path is None or len(sp) < len(shortest_path):
                    shortest_path = sp

        return shortest_path


    


routes = [
          ('Mumbai', 'Paris'),
          ('Mumbai', 'Dubai'),
          ('Paris', 'Dubai'),
          ('Paris', 'New York'),
          ('Dubai', 'New York'),
          ('New York', 'Toronto')]

route_graph = Graph(routes)

start = 'Paris'
end = 'New York'

print(f'Paths between {start} and {end}: ', route_graph.get_paths(start,end))
print(f'Shortest path between {start} and {end}: ', route_graph.get_shortest_path(start,end))


In [ ]:
# puzzle_input = '''light red bags contain 1 bright white bag, 2 muted yellow bags.
# dark orange bags contain 3 bright white bags, 4 muted yellow bags.
# bright white bags contain 1 shiny gold bag.
# muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.
# shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.
# dark olive bags contain 3 faded blue bags, 4 dotted black bags.
# vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.
# faded blue bags contain no other bags.
# dotted black bags contain no other bags.'''

puzzle_processing = [i for i in puzzle_input.splitlines()]
puzzle_processing = [i.replace('.','').replace('no other bags','').replace(' bags','').replace(' bag','').split(' contain ') for i in puzzle_processing]
puzzle_processing = {key: value.split(', ') for (key, value) in puzzle_processing}

rules = {}

for key, val in puzzle_processing.items():
    if val == ['']:
        continue
    if len(val) > 1:
        val = [i.split(' ', 1) for i in val]
    rules[key] = val

answer = 0
for rule in rules:
    bags = [rule]
    while len(bags) > 0:
        if bags[0] in rules:
            for newbag in rules[bags[0]]:
                if newbag[1] not in bags:
                    bags.append(newbag[1])
        del bags[0]
        if 'shiny gold' in bags:
            answer += 1
            break
print(answer)

# [['light', 'red', 'bags', 'contain', '1', 'bright', 'white', 'bag,', '2', 'muted', 'yellow', 'bags.']]
# [['dark', 'orange', 'bags', 'contain', '3', 'bright', 'white', 'bags,', '4', 'muted', 'yellow', 'bags.']]
# [['bright', 'white', 'bags', 'contain', '1', 'shiny', 'gold', 'bag.']]
# [['muted', 'yellow', 'bags', 'contain', '2', 'shiny', 'gold', 'bags,', '9', 'faded', 'blue', 'bags.']]
# [['shiny', 'gold', 'bags', 'contain', '1', 'dark', 'olive', 'bag,', '2', 'vibrant', 'plum', 'bags.']]
# [['dark', 'olive', 'bags', 'contain', '3', 'faded', 'blue', 'bags,', '4', 'dotted', 'black', 'bags.']]
# [['vibrant', 'plum', 'bags', 'contain', '5', 'faded', 'blue', 'bags,', '6', 'dotted', 'black', 'bags.']]
# [['faded', 'blue', 'bags', 'contain', 'no', 'other', 'bags.']]
# [['dotted', 'black', 'bags', 'contain', 'no', 'other', 'bags.']]

In [ ]:
for i in puzzle_processing:
    bag = ' '.join(i[0:2])

    if i[4] == 'no':
        continue
    bag_contents_1 = ' '.join(i[4:7])

    if len(i) != 12:
        graph_dict[bag] = [bag_contents_1.split(' ', 1)]
        continue

    bag_contents_2 = ' '.join(i[8:11])
    graph_dict[bag] = [bag_contents_1.split(' ', 1), bag_contents_2.split(' ', 1)]

In [ ]:
#after parsing the input to a tree, create a recursive function that returns true if the bag or any of its children (recursively) contain the shiny bag.
#Also in part 2, count the bag itself and its children recursively.

#1) the shiny gold bag is directly in its contents, or 2) one of the bags in its contents _contains the shiny gold one_.

# NEED TO LEARN
# graph traversal / tree walking

In [ ]:
class Graph:
    def __init__(self, edges):
        self.edges = edges
        self.graph_dict = {}
        for start, end in self.edges:
            if start in self.graph_dict:
                self.graph_dict[start].append(end)
            else:
                self.graph_dict[start] = [end]
        print("Graph dict:", self.graph_dict)

    def get_paths(self, start, end, path=[]):
        path = path + [start]

        if start == end:
            return [path]
        
        if start not in self.graph_dict:
            return []
        
        paths = []

        for node in self.graph_dict[start]:
            if node not in path:
                new_paths = self.get_paths(node, end, path) 
                for p in new_paths:
                    paths.append(p)
        
        return paths

    def get_shortest_path(self, start, end, path=[]):
        path = path + [start]

        if start == end:
            return path

        if start not in self.graph_dict:
            return None

        shortest_path = None
        for node in self.graph_dict[start]:
            sp = self.get_shortest_path(node, end, path)
            if sp:
                if shortest_path is None or len(sp) < len(shortest_path):
                    shortest_path = sp

        return shortest_path


routes = [
          ('Mumbai', 'Paris'),
          ('Mumbai', 'Dubai'),
          ('Paris', 'Dubai'),
          ('Paris', 'New York'),
          ('Dubai', 'New York'),
          ('New York', 'Toronto')]

route_graph = Graph(routes)

start = 'Paris'
end = 'New York'

print(f'Paths between {start} and {end}: ', route_graph.get_paths(start,end))
print(f'Shortest path between {start} and {end}: ', route_graph.get_shortest_path(start,end))


In [ ]:
with open('/content/drive/MyDrive/advent_of_code/day7.txt', 'r') as file:
    puzzle_input = file.read()


def buildDB():
    db = dict()
    for l in puzzle_input.split("\n"):
        if l != '':
            l = l.replace(',','').split()
            key = "{} {}".format(l[0],l[1])
            t = list()
            l = l[4:]
            if (l != ['no','other','bags.']):
                for i in range(len(l)):
                    if(i % 4 == 0):
                        t.append((int(l[i]),"{} {}".format(l[i+1],l[i+2])))
                if (key not in db):
                    db[key] = t
    return db

db = buildDB()
print(db)
def part1():
    search = ['shiny gold']
    outer,start = set(),0
    while True:
        for i in db.keys(): #
            for j in db[i]:
                if (j[1] in outer or j[1] in search):
                    outer.add(i)
        if (len(outer) > start):
            start = len(outer)
        else:
            break
    print(len(outer))

def part2():
    def getContent(bag):
        c = 1
        if (bag in db):
            for i in db[bag]:
                c += i[0]*getContent(i[1])
        return c
    print(getContent('shiny gold')-1)

part1()
part2()

In [ ]:
import itertools
import copy
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt

print(db)

# for i,v in db.items():
#     for j in v:
#         print(i, j)
#df = pd.DataFrame(db)

# Day 8

In [ ]:
# # puzzle_input = '''nop +0
# # acc +1
# # jmp +4
# # acc +3
# # jmp -3
# # acc -99
# # acc +1
# # jmp -4
# # acc +6'''


# with open('day8.txt', 'r') as file:
#     puzzle_input = file.read()

# puzzle_processing = [i.split(' ') for i in puzzle_input.split('\n')]
# puzzle_processing = [[i[0],int(i[1])] for i in puzzle_processing]

# swappable_indices = [i for i in range(len(puzzle_processing)-1) if puzzle_processing[i][0] in ('jmp', 'nop')]


# def can_code_complete(code_lines, swap_position=None):
#         accumulator, pointer, line_register = 0, 0, {}
#         while True:
#                 if pointer == len(code_lines):
#                         return True, accumulator
#                 elif pointer in line_register:
#                         return False, accumulator
#                 line_register[pointer] = 1 if pointer not in line_register else line_register[pointer] + 1
#                 instruction = code_lines[pointer][0] if pointer != swap_position else ("nop" if code_lines[pointer][0] == "jmp" else "jmp")
#                 if instruction == "acc":
#                         accumulator += code_lines[pointer][1]
#                 elif instruction == "jmp":
#                         pointer += code_lines[pointer][1]
#                         continue
#                 pointer += 1


# print(can_code_complete(puzzle_processing)[1])
# x = []
# for i in swappable_indices:
#     l = can_code_complete(puzzle_input, swap_position=i)[0]
#     if l is True:
#         x.append(l[1])


# x = [l[1] for index in swappable_indices if (l := can_code_complete(code_lines, swap_position=index))[0] is True][0]

# # code_lines = [((l := line.split(" "))[0], int(l[1])) for line in code.split("\n")]
# # accumulator, pointer, line_register = 0, 0, {}
# swappable_indices = [index for index in range(len(code_lines)-1) if code_lines[index][0] in ("jmp", "nop")]
# print("Part1: accumulator value at first loop start: {}".format(can_code_complete(code_lines)[1]))

In [ ]:
input_list = []

with open('/content/drive/MyDrive/advent_of_code/day8.txt', 'r') as reader:
    for line in reader:
        this_line = str(line.rstrip()).split()
        input_list.append([this_line[0], int(this_line[1])])


def run_program(this_prog, flipped_index):
    accumulator = 0
    current_index = 0
    visited_indices = []

    while True:
        try:
            # Pulling out the operation and argument because nested lists are ugly
            # also we can change the operation without worrying about modifying the list
            operation = this_prog[current_index][0]
            argument = this_prog[current_index][1]
        except IndexError:  # We have successfully terminated the program. Hooray!
            print(f"End of program after flipping index {flipped_index}")
            return accumulator

        if current_index in visited_indices:
            return None  # Nope on outta here -- we got an infinite loop!
        visited_indices.append(current_index)

        # Flip the operation if we're at the target index
        if current_index == flipped_index:
            if operation == 'jmp':
                operation = 'nop'
            elif operation == 'nop':
                operation = 'jmp'

        # Carry out the operations
        if operation == 'acc':
            accumulator += argument
            current_index += 1
        elif operation == 'jmp':
            current_index += argument
        elif operation == 'nop':
            current_index += 1
        else:
            print(f"Unknown argument {argument} found at index {current_index}. Aborting.")
            break

# Gonna loop through each line, and when we find a jmp or an nop, run the program with it "flipped"
i = 0
while True:
    if input_list[i][0] == 'acc':   # No need to "flip" the acc operation, just move right along.
        i += 1
    else:
        result = run_program(input_list, i)
        if result is not None:  # None is returned if its an infinite loop
            print(f"Part 2: {result}")
            break
        else:
            i += 1

# Day 9

In [ ]:
# Part 1
from itertools import combinations

with open('/content/drive/MyDrive/advent_of_code/day9.txt', 'r') as file:
     puzzle_input = file.read()

puzzle_processing = [int(i) for i in puzzle_input.splitlines()]

for idx, val in enumerate(puzzle_processing[25:]):
    summed_pairs = [sum(pair) for pair in combinations(puzzle_processing[idx:idx+25], 2)]

    if val not in summed_pairs:
        print(val)

In [ ]:
# Part 2
class Found(Exception): pass

target = 26134589
step = 2

while True:
    try:
        for idx, val in enumerate(puzzle_processing):
            contiguous_set = puzzle_processing[idx:step]
            if sum(contiguous_set) == target:
                raise Found
        step += 1
    except Found:
        print(max(contiguous_set) + min(contiguous_set))
        break

In [ ]:
# Learning 1
input = list(map(int, open("input/2020/input09.txt", 'r')))
n = len(input)
part1 = input[next(i for i in range(25, n) if all(input[x] + y != input[i] for x in range(i-25, i-1) for y in input[x+1:i]))]
part2 = next(x for x in (next(input[i:j] for j in range(i+1, n) if sum(input[i:j]) >= part1) for i in range(n)) if sum(x) == part1)
print(part1)
print(min(part2) + max(part2))

# js https://davidlozzi.com/2020/12/09/advent-of-code-day-9/
# https://dev.to/qviper/advent-of-code-2020-python-solution-day-9-1knj
# https://dev.to/rpalo/advent-of-code-2020-solution-megathread-day-9-encoding-error-594o

In [ ]:
input = list(map(int, open("/content/drive/MyDrive/advent_of_code/day9.txt", 'r')))
n = len(input)
part1 = input[next(i for i in range(25, n) if all(input[x] + y != input[i] for x in range(i-25, i-1) for y in input[x+1:i]))]
part2 = next(x for x in (next(input[i:j] for j in range(i+1, n) if sum(input[i:j]) >= part1) for i in range(n)) if sum(x) == part1)
print(part1)
print(min(part2) + max(part2))


# Day 10 (review)

In [ ]:
pi = '''16
10
15
5
1
11
7
19
6
12
4
'''

pp = list(map(int, pi.splitlines()))
pp.append(0)
pp.append(max(pp) + 3)
pp.sort()
print(pp)

m = max(pp)
print(m)

#low_voltage = filter(lambda x: x <= 3, pp)
low_voltage = [x for x in pp if x <= 3]
print(low_voltage)
new = []

In [ ]:
Assume jolts[] is a vector with the input, plus 0 and max(input)+3
Assume jolts[] is sorted in ascending order
dp[0] = 1
for i = 1 to len(jolts)-1:
    dp[i] = 0
    j = i-1
    while jolts[i] - jolts[j] <= 3:
        dp[i] += dp[j]
        --j
print the last element of dp (i.e., dp[len(dp)-1])


Assume jolts[] is a vector with the input, plus 0 and max(input)+3
Assume jolts[] is sorted in ascending order
dp[0] = 1
sum = 1
j = 0
for i = 1 to len(jolts)-1:
    while jolts[i] - jolts[j] > 3:
        sum -= dp[j]
        ++j
    dp[i] = sum
    sum += dp[i]
print the last element of dp (i.e., dp[len(dp)-1])


In [ ]:

local joltages = {0} -- given input joltages; 0 is added first for the charging outlet
local diff1 = 0
local diff3 = 1 -- starts from 1 as our device's adapter has a built-in joltage of highest +3

-- read the input
for i in io.lines(input) do
    table.insert(joltages, tonumber(i))
end

-- sort the joltages
table.sort(joltages)

-- iterate over all joltages
for i = 1, (#joltages - 1) do
    -- if the difference is 1
    if joltages[i + 1] - joltages[i] == 1 then diff1 = diff1 + 1
    -- else if the difference is 3
    elseif joltages[i + 1] - joltages[i] == 3 then diff3 = diff3 + 1 end
end

print(diff1 * diff3)


In [ ]:
example = '''16
10
15
5
1
11
7
19
6
12
4
'''


joltages = list(map(int, open('/content/drive/MyDrive/advent_of_code/day10.txt', 'r')))
joltages.append(0)

diff1 = 0 # 1 jolt differences cnt
diff3 = 1 # 3 jolt differences cnt
joltages.sort()

for i in range(len(joltages)-1):
    if joltages[i+1] - joltages[i] == 1:
        diff1 = diff1 + 1
    elif joltages[i+1] - joltages[i] == 3:
        diff3 = diff3 + 1

print(diff1 * diff3)

In [ ]:
inp_file = open('/content/drive/MyDrive/advent_of_code/day10.txt')
adapters = []
for line in inp_file:
    adapters.append(int(line))

adapters.append(0)
adapters.sort()
adapters.append(adapters[-1] + 3)


class Graph:
    def __init__(self):
        self.adj_list = []

    def add_edge(self, from_vertex, to_vertex):
        self.adj_list[from_vertex].append(to_vertex)
    
    def add_vertex(self, vertex):
        self.adj_list.append(vertex)

    def __repr__(self):
        return str(self.adj_list)

    def __str__(self):
        return str(self.adj_list)


def part1():
    diffs = [0, 0, 0]

    for i in range(1, len(adapters)):
        diffs[adapters[i] - adapters[i-1] - 1] += 1

    return diffs[0] * diffs[2]



def part2():
    graph = Graph()
    current_vertex = 0
    for i in range(len(adapters)):
        new_edges = []
        for j in range(1,4):
            if i+j < len(adapters) and adapters[i+j] - adapters[i] <= 3:
                new_edges.append(current_vertex+j)

        if len(new_edges) == 1 and (len(graph.adj_list) == 0 or len(graph.adj_list[-1]) == 1):
            continue
        else:
            graph.add_vertex(new_edges)
            current_vertex += 1

    # Get the number of edges at each vertex
    num_of_edges = [len(graph.adj_list[i]) for i in range(len(graph.adj_list))]

    # Start from the second to last edge
    for i in range(len(graph.adj_list)-2, 0, -1):
        for j in range(1, 4):
            # If the current vertex is in a previous vertex
            if i-j >= 0 and i in graph.adj_list[i-j]:
                # Increase the number of edges in the previous vertex
                num_of_edges[i-j] += num_of_edges[i]
                # Remove an edge (because we're essentially doing a substitution)
                num_of_edges[i-j] -= 1
                

    return num_of_edges[0]


print (part1())
print (part2())

# Day 11

In [ ]:
import sys
from typing import Dict, Set, List, Tuple

def parse_seats(lines: List[str]) -> Dict[Tuple[int, int], Set[Tuple[int, int]]]:
    # find all seats in the input, and their connections.
    w, h = len(lines[0]), len(lines)
    seats = {(x, y): {(x, y)} for x in range(w) for y in range(h) if lines[y][x] != '.'}
    for path in generate_paths(w, h):
        previous = None
        for current in path:
            if current in seats:
                if previous:
                    seats[previous].add(current)
                    seats[current].add(previous)
                previous = current
    return seats

def generate_paths(w: int, h: int):
    # to the right
    for y in range(0, h):
        yield ((x, y) for x in range(w))
    # down
    for x in range(0, w):
        yield ((x, y) for y in range(h))
    # down and to the right
    for x0 in range(-h + 1, w):
        yield ((x0 + y, y) for y in range(max(0, -x0), min(h, w - x0)))
    # down and to the left
    for x0 in range(0, w + h):
        yield ((x0 - y, y) for y in range(max(0, x0 - w + 1), min(h, x0 + 1)))

def solve(seats: Dict[Tuple[int, int], Set[Tuple[int, int]]], n: int):
    # assume that all seats are free at the beginning.
    free = {seat for seat, visible in seats.items() if len(visible) > n} # after 2 iterations
    while True:
        # visible = {seat, *neighbours}
        changed = set(seat for seat, visible in seats.items() if visible <= free or seat not in free and len(visible - free) > n)
        if not changed:
            break
        free ^= changed
    return len(seats) - len(free)


lines = [line.rstrip() for line in open('/content/drive/MyDrive/advent_of_code/day11.txt', 'r')]
seats_part2 = parse_seats(lines)
seats_part1 = {(i, j): {(p, q) for (p, q) in visible if max(abs(i-p), abs(j-q)) < 2} for (i, j), visible in seats_part2.items()}
print(f'Part 1: {solve(seats_part1, 4)}')
print(f'Part 2: {solve(seats_part2, 5)}')

In [ ]:
from itertools import permutations

def words(letters):
    for n in range(1, len(letters)+1):
        yield from map(''.join, permutations(letters, len(letters)))


word = '12'

for i in words(word):
    print(i)

In [ ]:
x = 'asfs-afsf'
x.partition('-')